<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/4_2the_image_classification_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d2l==1.0.0-alpha1.post0

In [7]:
%matplotlib inline
import time
import torch
import torchvision
from torchvision import transforms
from matplotlib_inline import backend_inline

In [8]:
def use_svg_display():
  """Use the svg (Scalable Vector Graphics) format to display a plot in Jupyter."""
  backend_inline.set_matplotlib_formats('svg')

use_svg_display()

## 1. Object-oriented Design for Implementation

In [ ]:
import numpy as np
from torch import nn
from d2l import torch as d2l

### 1.1 Utilities

In [10]:
def add_to_class(Class):
  def wrapper(obj):
    setattr(Class, obj.__name__, obj)
  return wrapper

In [11]:
class A:
  def __init__(self):
    self.b = 1

a = A()

In [12]:
@add_to_class(A)
def do(self):
  print('Class attribute "b" is', self.b)

a.do()

Class attribute "b" is 1
